# 2 - Easy Wins 1
##### **Author: Adam Gatt**

Later notebooks will cover more advance topics but the next two will aim to show off simple new language features that you should be able to immediately benefit from with little/no trade-off. Many revolve around enforcing correctness, to help remove some classes of bugs before they have a chance to happen.

The [clang-tidy](https://clang.llvm.org/extra/clang-tidy/) linter will be able to make suggestions about many of these language features, pointing out situation where they can be used. With some care and manual review, you can even run the linter over an entire existing codebase and have the suggestions applied automatically.

## override

How come the duck below fails to realise it can quack?

In [1]:
#include <iostream>

class Bird {
    public:
    virtual void makeNoise() const {
        std::cout << "Chirp!" << std::endl;
    }
};

class Duck : public Bird {
    public:
    virtual void makeNoise() {
        std:: cout << "Quack!" << std::endl;
    }
};

Bird* daffy = new Duck();

daffy->makeNoise();

Chirp!


It's because the two `makeNoise()` functions aren't the same! Because one is const and the other isn't, they count as having different signatures. This means the derived class isn't overriding the makeNoise in the base class; the two methods don't occupy the same entry in the vtable. 

Constness is a subtle way this bug can occur, but it can also be caused by a typo in the method name or differences in parameter types.

Because both the base and overridden methods use the same syntax (beginning `virtual` annotation), the compiler doesn't have a clear way to know that `Duck::makeNoise` is meant to be an overridden method instead of starting its own base virtual method. `override` is a new annotation to add to the overriding method and make it clear to the compiler. When a method is marked `override` the compiler will search for a matching base virtual method to ensure it exists, and will throw a compilation error if one cannot be found.

As an added benefit it also makes the intention clear to the human reader.

In [2]:
class Turkey : public Bird {
    public:
    virtual void makeNoise() override {
        std:: cout << "Gobble!" << std::endl;
    }
};

input_line_9:3:18: error: 'makeNoise' marked 'override' but does not override any member functions
    virtual void makeNoise() override {
                 ^


Interpreter Error: 

## nullptr

`nullptr` is a Modern C++ keywork for the null reference, where in the past you might have used `0` or `NULL` (often #defined to be 0). The advantage of nullptr is that it has its own type (`std::nullptr_t`), whereas the `int` type of NULL can allow for subtle bugs or ambiguities, especially in constructors and overloaded functions. In the best-case scenario the compiler will notice the ambiguity with a warning or error, in the worst-case you may have a subtle logic error.

In [3]:
class Colour {    
    public:
    int rgbCode;
    
    // Specify the rgb code directly
    Colour(int rgbCode) : rgbCode(rgbCode) { }
    
    // Copy the rgb code from another Colour
    Colour(const Colour* cloneSrc) {
        if (cloneSrc != NULL) {
            rgbCode = cloneSrc->rgbCode;
        }
    }
}

In [4]:
// Which constructor will be called?

Colour col1(NULL);

input_line_11:3:8: error: call to constructor of '__cling_N54::Colour' is ambiguous
Colour col1(NULL);
       ^    ~~~~
input_line_10:6:5: note: candidate constructor
    Colour(int rgbCode) : rgbCode(rgbCode) { }
    ^
input_line_10:9:5: note: candidate constructor
    Colour(const Colour* cloneSrc) {
    ^
input_line_10:1:7: note: candidate is the implicit copy constructor
class Colour {    
      ^
input_line_10:1:7: note: candidate is the implicit move constructor


Interpreter Error: 

In [5]:
Colour col2(nullptr);

Notice the constructor executing correctly instead of trying to dereference nullptr. This is because the guard clause `if (cloneSrc != NULL)` still performed its job. The new keyword enjoys strong interoperability with existing legacy code as the rules for comparing nullptr and NULL have been well-defined to produce intuitive results.

In [1]:
bool comparisons[] {
    nullptr == nullptr,
    nullptr != nullptr,
    nullptr == NULL,
    nullptr != NULL,
    nullptr == 0,
    nullptr != 0,
    nullptr ? true : false
};

comparisons

:13: warning: comparison between NULL and non-pointer ('nullptr_t' and NULL) [-Wnull-arithmetic]
    nullptr == NULL,
    ~~~~~~~ ^  ~~~~
input_line_7:6:13: warning: comparison between NULL and non-pointer ('nullptr_t' and NULL) [-Wnull-arithmetic]
    nullptr != NULL,
    ~~~~~~~ ^  ~~~~
input_line_7:9:5: warning: implicit conversion of nullptr constant to 'bool' [-Wnull-conversion]
    nullptr ? true : false
    ^~~~~~~ ~
    false


{ true, false, true, false, true, false, false }

Another example: when creating a nullable reference to a _Line_ object we accidentally leave out the asterisk and get unexpected results that will throw no errors. We would think that `NULL`, an int, shouldn't be assignable to a Line object. But despite the `=` character in the bottom-most line, the operation that actually takes place here is _copy initialisation_ rather than assignment. As such the constructor is called with the argument `NULL`, which the compiler happily treats as the int that it is.

In [2]:
#include <iostream>

class Line {
    private:
    int thickness;
    
    public:
    Line(int thickness) : thickness(thickness) {
        std::cout << "Created line with thickness " << thickness;
    }
};

Line myLine = NULL;

Created line with thickness 0

## explicit (pre-11)

`explicit` 

In [ ]:
class Factor {
    public:
    Factor(double value)
        : value(value) { }
    
    double apply(double input) {
        return input * value;
    }
    
    private:
    double value;
}

In [ ]:
Factor myFactor = 1/5;

myFactor.apply(100)

In [ ]:
class Factor {
    public:
    // Explicit, no implicit type conversion for copy-initialisation
    explicit Factor(double value)
        : value(value) { }
    
    double apply(double input) {
        return input * value;
    }
    
    private:
    double value;
};

Factor myFactor = 1/5;

myFactor.apply(100)

## Scoped enums